In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import transformers
#from transformers import AutoTokenizer
from transformers import DistilBertForTokenClassification
from torch.optim import AdamW
import torch
import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
import re

from transformers import AutoTokenizer, AutoModelForMaskedLM

#import part

In [2]:
N = 43821 #columns of dataset
df = pd.read_csv("NER_dataset.csv", encoding="cp949").sample(frac=1)[:N] #load dataset

#change field name
df.rename(columns = {'text':'sentence', 'labels':'tags'}, inplace = True)

#split train, dev, test data (dev data is used for cross validation)
df_train, df_dev, df_test = np.split(df.sample(frac=1, random_state=0), [int(.8 * len(df)), int(.9 * len(df))])

In [3]:
df.head()
#show head part of dataset

,sentence,tags
22920,ADMISSION DATE :,O O O
10027,SOCIAL HISTORY :,O O O
8066,This is the baby 's discharge feedings and he ...,O O O O O O O O O O O O O O O O O O O O O O O ...
32300,"Abd : Soft , NT/ND , NABS",O O O O B-PR O O
30688,ABD : soft ND/NT +BS,O O O B-PR O


In [4]:
df.tail()
#show tail part of dataset

,sentence,tags
11534,"no masses , neck revealed lymphadenopathy .",O B-PR O O O B-PR O
955,Post thoracotomy syndrome .,B-PR I-PR I-PR O
21815,"For possible COPD , pt was started on steroids...",O O B-PR O O O O O B-TR O B-TR O O O
28621,The patient refused an LP .,O O O B-TE I-TE O
31980,2012-06-07 09:55 PM BLOOD PT - 12.8 PTT - 25.7...,O O O B-TE B-TE O O B-TE O O B-TE O O


In [5]:
# tansfrom label to list (delimiter is " ")
labels = [i.split() for i in df['tags'].values.tolist()]

# count the number of label
unique_labels = set()

for lb in labels:
    [unique_labels.add(i) for i in lb if i not in unique_labels]
 
print(unique_labels)
# {'B-TE', 'I-TR', 'I-PR', 'I-TE', 'O', 'B-TR', 'B-PR'}

# mappingg unique_lables to their id
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}
print(labels_to_ids)
# {'B-PR': 0, 'B-TE': 1, 'B-TR': 2, 'I-PR': 3, 'I-TE': 4, 'I-TR': 5, 'O': 6}

{'O', 'I-TR', 'I-PR', 'B-TR', 'I-TE', 'B-PR', 'B-TE'}
{'B-PR': 0, 'B-TE': 1, 'B-TR': 2, 'I-PR': 3, 'I-TE': 4, 'I-TR': 5, 'O': 6}


In [6]:
#main class for doing NER with Distilbert

class DistilbertNER(nn.Module):
  
    def __init__(self, tokens_dim): #constructor 
        super(DistilbertNER,self).__init__()
    
        if type(tokens_dim) != int:
            raise TypeError('tokens_dim should be an integer')

        if tokens_dim <= 0:
              raise ValueError('Classification layer dimension should be at least 1')

        self.pretrained = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels = tokens_dim) #for using distilBERT model
        #self.pretrained = AutoModelForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2", num_labels = tokens_dim) #for using bioBERT model
        #set the output of each token classifier = unique_lables

    def forward(self, input_ids, attention_mask, labels = None): #compute loss

        if labels == None:
            out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask )
            return out #return without label

        out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask , labels = labels)
        return out #return with label

In [7]:
#generalized function for get information from dataset

class NerDataset(torch.utils.data.Dataset):
  
  def __init__(self, df):
    if not isinstance(df, pd.DataFrame):
      raise TypeError('Input should be a dataframe')
    
    if "tags" not in df.columns or "sentence" not in df.columns:
      raise ValueError("Dataframe should contain 'tags' and 'sentence' columns")
    
    tags_list = [i.split() for i in df["tags"].values.tolist()]
    texts = df["sentence"].values.tolist()
    
    #for change float(nan) -> string("nan")
    i = 0
    for string in texts:
        i += 1
        if(isinstance(string, float)):
            texts[i - 1] = "nan"

    self.texts = [tokenizer(text, padding = "max_length", truncation = True, return_tensors = "pt") for text in texts]
    self.labels = [match_tokens_labels(text, tags) for text,tags in zip(self.texts, tags_list)]

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    batch_text = self.texts[idx]
    batch_labels = self.labels[idx]

    return batch_text, torch.LongTensor(batch_labels)

In [8]:
class MetricsTracking():
  """
  In order make the train loop lighter I define this class to track all the metrics that we are going to measure for our model.
  """
  def __init__(self):

    self.total_acc = 0
    self.total_f1 = 0
    self.total_precision = 0
    self.total_recall = 0

  def update(self, predictions, labels , ignore_token = -100):
    '''
    Call this function every time you need to update your metrics.
    Where in the train there was a -100, were additional token that we dont want to label, so remove them.
    If we flatten the batch its easier to access the indexed = -100
    '''  
    predictions = predictions.flatten()
    labels = labels.flatten()
    
    predictions = predictions[labels != ignore_token]
    labels = labels[labels != ignore_token]

    predictions = predictions.to("cpu")
    labels = labels.to("cpu")

    acc = accuracy_score(labels,predictions)
    f1 = f1_score(labels, predictions, average = "macro")
    precision = precision_score(labels, predictions, average = "macro")
    recall = recall_score(labels, predictions, average = "macro")

    self.total_acc  += acc
    self.total_f1 += f1
    self.total_precision += precision
    self.total_recall  += recall

  def return_avg_metrics(self,data_loader_size):
    n = data_loader_size
    metrics = {
        "acc": round(self.total_acc / n ,3), 
        "f1": round(self.total_f1 / n, 3), 
        "precision" : round(self.total_precision / n, 3), 
        "recall": round(self.total_recall / n, 3)
          }
    return metrics

In [9]:
#create label

def tags_2_labels(tags : str, tag2idx : dict):
  return [tag2idx[tag] if tag in tag2idx else unseen_label for tag in tags.split()] 

In [10]:
#map words to tag

def tags_mapping(tags_series : pd.Series):
  if not isinstance(tags_series, pd.Series):
      raise TypeError('Input should be a padas Series')

  unique_tags = set()
  
  for tag_list in df_train["tags"]:
    for tag in tag_list.split():
      unique_tags.add(tag)

  tag2idx = {k:v for v,k in enumerate(sorted(unique_tags))}
  idx2tag = {k:v for v,k in tag2idx.items()}

  unseen_label = tag2idx["O"]

  return tag2idx, idx2tag, unseen_label, unique_tags

In [11]:
#-100 means CLS or PAD

def match_tokens_labels(tokenized_input, tags, ignore_token = -100):
        word_ids = tokenized_input.word_ids()

        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:

            if word_idx is None:
                label_ids.append(ignore_token)

            else :
                try:
                  reference_tag = tags[word_idx]
                  label_ids.append(tag2idx[reference_tag])
                except:
                  label_ids.append(ignore_token)
            
            previous_word_idx = word_idx

        return label_ids

In [12]:
#train & evaluation function

def train_loop(model, train_dataset, dev_dataset, optimizer,  batch_size, epochs):
  
  train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
  dev_dataloader = DataLoader(dev_dataset, batch_size = batch_size, shuffle = True)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)

  for epoch in range(epochs) : 
    
    train_metrics = MetricsTracking()
    total_loss_train = 0

    model.train() #core function for train

    for train_data, train_label in tqdm(train_dataloader):

      train_label = train_label.to(device)

      mask = train_data['attention_mask'].squeeze(1).to(device)
      input_id = train_data['input_ids'].squeeze(1).to(device)

      optimizer.zero_grad()
      
      output = model(input_id, mask, train_label)
      loss, logits = output.loss, output.logits
      predictions = logits.argmax(dim= -1) 

      train_metrics.update(predictions, train_label)
      total_loss_train += loss.item()

      loss.backward()
      optimizer.step()
    
    model.eval() #core function for evaluation

    dev_metrics = MetricsTracking()
    total_loss_dev = 0
    
    with torch.no_grad():
      for dev_data, dev_label in dev_dataloader:

        dev_label = dev_label.to(device)

        mask = dev_data['attention_mask'].squeeze(1).to(device)
        input_id = dev_data['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask, dev_label)
        loss, logits = output.loss, output.logits

        predictions = logits.argmax(dim= -1)     

        dev_metrics.update(predictions, dev_label)
        total_loss_dev += loss.item()
    
    train_results = train_metrics.return_avg_metrics(len(train_dataloader))
    dev_results = dev_metrics.return_avg_metrics(len(dev_dataloader))

    print(f"TRAIN \nLoss: {total_loss_train / len(train_dataset)} \nMetrics {train_results}\n" ) 
    print(f"VALIDATION \nLoss {total_loss_dev / len(dev_dataset)} \nMetrics{dev_results}\n" )   

In [13]:
#create mapp between label and tag
tag2idx, idx2tag , unseen_label, unique_tags = tags_mapping(df_train["tags"])

#change label to tag (surplus label will be changed to "O" tag)
for df in [df_train, df_dev, df_test]:
  df["labels"] = df["tags"].apply(lambda tags : tags_2_labels(tags, tag2idx))

In [14]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #needed to useing distilBERT model
#tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2", model_max_length = 512) #needed to useing bioBERT model

In [15]:
text = df_train["sentence"].values.tolist()

#for change float(nan) -> string("nan")

i = 0
for string in text:
    i += 1
    if(isinstance(string, float)):
        text[i - 1] = "nan"

text_tokenized = tokenizer(text, padding = "max_length", truncation = True, return_tensors = "pt")
#text_tokenized = text_tokenized.remove_columns(books_dataset["train"].column_names) #needed to useing bioBERT tokenizer

#map tokens to corresponding words
word_ids = text_tokenized.word_ids()


In [16]:
model = DistilbertNER(len(unique_tags))
learn = False

#determine whether new train & learn is needed or not

try :
    model = torch.load("NER_model", map_location=torch.device('cpu'))
except FileNotFoundError as e : 
    print("MODEL is not exist so new MODEL will be created")
    learn = True

model.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

MODEL is not exist so new MODEL will be created


DistilbertNER(
  (pretrained): DistilBertForTokenClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn):

In [ ]:
#set the hyperparameters

train_dataset = NerDataset(df_train)
dev_dataset = NerDataset(df_dev)

lr = 1e-2
optimizer = SGD(model.parameters(), lr=lr, momentum = 0.9)  

#MAIN
parameters = {
    "model": model,
    "train_dataset": train_dataset,
    "dev_dataset" : dev_dataset,
    "optimizer" : optimizer,
    "batch_size" : 16,
    "epochs" : 5
}

if learn == True: #do train & test if NER_model not exist
    train_loop(**parameters)

100%|██████████████████████████████████████████████████████████████████████████████| 2191/2191 [07:38<00:00,  4.78it/s]


TRAIN 
Loss: 0.02761397135081828 
Metrics {'acc': 0.85, 'f1': 0.638, 'precision': 0.716, 'recall': 0.631}

VALIDATION 
Loss 0.020843860015718564 
Metrics{'acc': 0.886, 'f1': 0.716, 'precision': 0.793, 'recall': 0.704}



100%|██████████████████████████████████████████████████████████████████████████████| 2191/2191 [07:34<00:00,  4.82it/s]


TRAIN 
Loss: 0.01842606408548206 
Metrics {'acc': 0.897, 'f1': 0.758, 'precision': 0.82, 'recall': 0.748}

VALIDATION 
Loss 0.015522936243004501 
Metrics{'acc': 0.914, 'f1': 0.794, 'precision': 0.843, 'recall': 0.785}



100%|██████████████████████████████████████████████████████████████████████████████| 2191/2191 [07:38<00:00,  4.78it/s]


TRAIN 
Loss: 0.014188383151187839 
Metrics {'acc': 0.919, 'f1': 0.813, 'precision': 0.861, 'recall': 0.805}

VALIDATION 
Loss 0.015683170521716 
Metrics{'acc': 0.914, 'f1': 0.803, 'precision': 0.839, 'recall': 0.807}



100%|██████████████████████████████████████████████████████████████████████████████| 2191/2191 [07:34<00:00,  4.82it/s]


TRAIN 
Loss: 0.011856908196162202 
Metrics {'acc': 0.931, 'f1': 0.842, 'precision': 0.883, 'recall': 0.834}

VALIDATION 
Loss 0.01612855273752466 
Metrics{'acc': 0.917, 'f1': 0.812, 'precision': 0.829, 'recall': 0.831}



 99%|█████████████████████████████████████████████████████████████████████████████▌| 2179/2191 [07:32<00:02,  4.76it/s]

In [ ]:
if learn == True: #do export the result of train if NER_model not exist
    torch.save(model, "NER_model")